# Vector Database CRUD Class

Este codigo incluye la clase VectorDatabase la cual contiene las funcionalidades CRUD y de creacion de la base de datos.

In [4]:
#Importacion de librerias
import sqlite3
import numpy as np
import matplotlib.pyplot as plt

# Vector Database with CRUD Operations
class VectorDatabase:
    def __init__(self, db_name):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self._create_table()

    def _create_table(self):
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS embeddings (
            id INTEGER PRIMARY KEY,
            name TEXT,
            embedding BLOB
        )
        """)
        self.conn.commit()

    def create(self, name, embedding):
        embedding_blob = np.array(embedding, dtype=np.float32).tobytes()
        self.cursor.execute(
            "INSERT INTO embeddings (name, embedding) VALUES (?, ?)",
            (name, embedding_blob)
        )
        self.conn.commit()
    
    def read(self, name):
        self.cursor.execute("SELECT * FROM embeddings WHERE name = ?", (name,))
        result = self.cursor.fetchone()
        if result:
            return result[0], result[1], np.frombuffer(result[2], dtype=np.float32)
        return None

    def update(self, name, new_embedding):
        embedding_blob = np.array(new_embedding, dtype=np.float32).tobytes()
        self.cursor.execute(
            "UPDATE embeddings SET embedding = ? WHERE name = ?",
            (embedding_blob, name)
        )
        self.conn.commit()

    def delete(self, name):
        self.cursor.execute("DELETE FROM embeddings WHERE name = ?", (name,))
        self.conn.commit()

    def close(self):
        self.conn.close()

## Example execution file

Este codigo sirve para probar la funcionalidad de la base de datos

In [5]:
db = VectorDatabase("vector_database.db")

db.create("vector_1", [1, 0, 0])
db.create("vector_2", [0, 1, 0])

print(db.read("vector_1"))

db.update("vector_1", [0.5, 0.5, 0])

db.delete("vector_2")